In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re
from datetime import datetime
import os
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from requests.exceptions import InvalidURL
from unidecode import unidecode

ModuleNotFoundError: No module named 'unidecode'

In [2]:
def extract_link(base_url):
    # Headers for request
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}
    page = 1
    all_links = []
    max_pages = 12  # Set maximum number of pages to navigate

    while page <= max_pages:
        url = f'{base_url}?p={page}'
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # New lines
        a_tags = soup.find_all('a', class_='product-item-link')
        all_links.extend([tag.get('href') for tag in a_tags])
        
        # Increment page number for the next iteration
        page += 1

    return all_links

In [3]:
def extract_data(all_links, start, end):
    data = []
    for link in all_links[start:end]:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}
        try:
            response = requests.get(link, headers=headers)
            response.raise_for_status()  # Raise an error for bad status codes
        except (requests.exceptions.RequestException, InvalidURL) as e:
            print(f"Skipping URL due to error: {link} - {e}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # image_link
        image_link_meta = soup.find('meta', attrs={'property': 'og:image'})
        image_link = image_link_meta['content'].strip() if image_link_meta else None

        # rating
        rating_element = soup.find('div', class_='rating-result')
        rating = rating_element.get('title').replace('%', '') if rating_element else None
        rating_count_element = soup.find('div', class_='reviews-actions')
        rating_count = ''.join(filter(str.isdigit, rating_count_element.get_text())) if rating_count_element else None

        # Price
        price_element = soup.find('span', class_='price-wrapper', attrs={'data-price-type': 'finalPrice'})
        price = price_element.get('data-price-amount') if price_element else None
        listing_price_element = soup.find('span', class_='price-wrapper', attrs={'data-price-type': 'oldPrice'})
        listing_price = listing_price_element.get('data-price-amount') if listing_price_element else None
        
        # SKU
        sku_element = soup.find('div', class_='product attribute sku')
        sku = sku_element.find('div', class_='value').get_text(strip=True) if sku_element else None

        # sku_name
        sku_name_meta = soup.find('meta', attrs={'property': 'og:title'})
        sku_name = sku_name_meta['content'].strip() if sku_name_meta else None

        # brand
        brand_element = soup.find('div', id='logo_name_brand')
        brand = brand_element.find('span', class_='brand-name').get_text(strip=True) if brand_element else None

        # status
        status_element = soup.select_one('.action.primary.tocart.action-tocart-sticky')
        status = status_element.get_text(strip=True) if status_element else None

        # voucher
        voucher_elements = soup.select('div.saving-voucher ul.saving-voucher-ul li.rule-desc-detail')
        vouchers = [voucher.get_text(strip=True) for voucher in voucher_elements]

        # gift
        gifts = []
        gift_elements = soup.select('div.product_info_promo')
        for gift in gift_elements:
            product_name = gift.find('a', class_='ampromo-product-name').text.strip()
            old_price = gift.find('span', class_='ampromo-product-oldPrice').text.strip().replace('₫', '').replace(',', '')
            gifts.append(f"{product_name} - {old_price}")

        # Extracting all info sections (e.g., Giới thiệu, Công dụng)
        info_sections = {}
        # Find all headers and corresponding content sections
        headers = soup.find_all('a', class_='data switch')
        content_sections = soup.find_all('div', class_='data item content')

        # Loop through both headers and contents
        for header_element, content_element in zip(headers, content_sections):
            header = header_element.get_text(strip=True)
            content = content_element.get_text(strip=True)
            info_sections[header] = content

        data.append({
            'url': link,
            'sku': sku,
            'image_link': image_link,
            'sku_name': sku_name,
            'rating': rating,
            'rating_count': rating_count,
            'price': price,
            'listing_price': listing_price,
            'brand': brand,
            'status': status,
            'vouchers': vouchers,
            'gifts': gifts,
            'info_sections': info_sections
        })
    return data

In [4]:
def save_data(data):
    df = pd.DataFrame(data)
    df['date'] = pd.Timestamp.now().normalize()

    df2 = df.copy()
    
    # Ensure `info_sections` is a dictionary, convert from string if necessary
    df2['info_sections'] = df2['info_sections'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    info_df = pd.json_normalize(df2['info_sections'])  # Converts dictionary into a DataFrame with keys as columns
    df2 = pd.concat([df2, info_df], axis=1)  # Concatenate the expanded info sections

    # Filepath for the CSV file
    file_path1 = 'guardian.csv'
    file_path2 = 'guardian_info.csv'

    # Append the new data to the existing CSV file
    if os.path.exists(file_path1):
        df.to_csv(file_path1, mode='a', index=False, header=False, encoding='utf-8-sig')
    else:
        df.to_csv(file_path1, index=False, encoding='utf-8-sig')

    if os.path.exists(file_path2):
        df2.to_csv(file_path2, mode='a', index=False, header=False, encoding='utf-8-sig')
    else:
        df2.to_csv(file_path2, index=False, encoding='utf-8-sig')
        
    return df2

# 1. Get all links

In [3]:
all_links = []
all_links.extend(extract_link('https://www.guardian.com.vn/cham-soc-da-mat/chong-nang.html'))

In [6]:
all_links.extend(extract_link('https://www.guardian.com.vn/cham-soc-da-mat/tay-trang.html'))

In [7]:
len(all_links)

265

In [8]:
all_links = list(dict.fromkeys(all_links))
len(all_links)

265

# 2. Extract data

In [ ]:
data = []
x = 0
y = 25
z = 25
data.extend(extract_data(all_links,x,y))



In [ ]:
import time
max_retries = 5   # number of retry attempts per iteration
retry_delay = 3   # seconds between retries

data = []

while x < len(all_links):  # or some stopping condition
    y = x + z

    attempt = 0
    while attempt < max_retries:
        try:
            data.extend(extract_data(all_links, x, y))
            print(f"Extracted: x={x}, y={y}")
            break  # exit retry loop on success
        except Exception as e:
            attempt += 1
            print(f"Error at x={x}, y={y} | Attempt {attempt}/{max_retries} | {e}")
            if attempt == max_retries:
                print("Skipping this batch and continuing")
            else:
                time.sleep(retry_delay)

    x = y  # move to next batch


In [10]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [11]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [12]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [13]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [14]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [15]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [16]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [17]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [18]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [19]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [20]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))
print(x,y) #300

275 300


In [21]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [22]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [23]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [24]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [25]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [26]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [27]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [28]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [29]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [30]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [31]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [32]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [33]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [34]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [35]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [36]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [37]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [38]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [39]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))
print(x,y)

750 775


In [40]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [41]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [42]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))
print(x,y)

825 850


In [43]:
x = x+z
y = y+z
data.extend(extract_data(all_links,x,y))

In [44]:
df1 = pd.DataFrame(data)
df1

,url,sku,image_link,sku_name,rating,rating_count,price,listing_price,brand,status,vouchers,gifts,info_sections
0,https://www.guardian.com.vn/kem-chong-nang-mon...,3030949,https://www.guardian.com.vn/media/catalog/prod...,KEM CHỐNG NẮNG BLISSBERRY DƯỠNG TRẮNG 60ML,0,0,379000,97000,BLISSBERRY,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Kem Chống Nắng Mỏng Nhẹ Dưỡng ...
1,https://www.guardian.com.vn/kem-chong-nang-l-o...,3027053,https://www.guardian.com.vn/media/catalog/prod...,Kem Chống Nắng L'Oreal Mỏng Nhẹ Bảo Vệ Tối Đa ...,95,4,299000,379000,L'ORÉAL,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Kem Chống Nắngthương hiệuL’Ore...
2,https://www.guardian.com.vn/gel-chong-nang-cap...,3025476,https://www.guardian.com.vn/media/catalog/prod...,"Gel Chống Nắng Cấp Ẩm, Nâng Tông Chiết Xuất Ý ...",80,1,143000,179000,REIHAKU HATOMUGI,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],"{'Giới Thiệu': 'Gel Chống NắngCấp Ẩm, Nâng Tôn..."
3,https://www.guardian.com.vn/kem-chong-nang-cho...,3024648,https://www.guardian.com.vn/media/catalog/prod...,Kem Chống Nắng Cho Da Nhạy Cảm Skin1004 Madaga...,95,8,249000,445000,SKIN1004,Thêm vào giỏ hàng,[VoucherG-DN- Giảm 60K cho đơn hàng có địa chỉ...,[[HÀNG TẶNG KHÔNG BÁN] Kem Chống Nắng Dạng Tuý...,{'Giới Thiệu': 'Kem Chống Nắng Cho Da Nhạy Cảm...
4,https://www.guardian.com.vn/kem-chong-nang-kie...,3016733,https://www.guardian.com.vn/media/catalog/prod...,Kem Chống Nắng Kiểm Soát Nhờn Không Màu La Roc...,93,3,365000,560000,LA ROCHE-POSAY,Thêm vào giỏ hàng,[VoucherG-DN- Giảm 60K cho đơn hàng có địa chỉ...,"[LA ROCHE-POSAY TÚI TÍM MELA B3 - 50000, [HÀNG...",{'Giới Thiệu': 'Kem Chống Nắng La Roche-Posay ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,https://www.guardian.com.vn/nuoc-tay-trang-pro...,3025187,https://www.guardian.com.vn/media/catalog/prod...,Nước Tẩy Trang Professional Clean Eye & Lip Re...,0,0,99000,229000,CATHY DOLL,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Nước Tẩy Trang Professional Cl...
261,https://www.guardian.com.vn/dau-tay-trang-lam-...,3027315,https://www.guardian.com.vn/media/catalog/prod...,Dầu Tẩy Trang Làm Sạch Cân Bằng Coboté 150Ml,0,0,220000,65000,COBOTÉ,Sắp về hàng,[],[],{'Giới Thiệu': 'THÀNH PHẦN - INGREDIENTS:Dầu d...
262,https://www.guardian.com.vn/khan-uot-tay-trang...,3028453,https://www.guardian.com.vn/media/catalog/prod...,KHĂN TẨY TRANG TẠO BỌT GUARDIAN AMINO ACID BUB...,0,0,69000,99000,GUARDIAN,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Khăn Ướt Tẩy Trang Giúp Làm Sạ...
263,https://www.guardian.com.vn/tay-trang-mi-judyd...,3030706,https://www.guardian.com.vn/media/catalog/prod...,TẨY TRANG MI JUDYDOLL MASCARA REMOVER - 5ML,0,0,167000,186000,JUDYDOLL,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,"[FG JUDYDOLL LUOC HOA MINI - 119000, [Hàng Tặn...",{'Giới Thiệu': 'Tẩy Trang Mi Judydoll Mascara ...


In [45]:
save_data(data)

,url,sku,image_link,sku_name,rating,rating_count,price,listing_price,brand,status,vouchers,gifts,info_sections,date,Giới Thiệu,Công Dụng,Thành Phần,Hướng Dẫn Sử Dụng,Thông tin thêm
0,https://www.guardian.com.vn/kem-chong-nang-mon...,3030949,https://www.guardian.com.vn/media/catalog/prod...,KEM CHỐNG NẮNG BLISSBERRY DƯỠNG TRẮNG 60ML,0,0,379000,97000,BLISSBERRY,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Kem Chống Nắng Mỏng Nhẹ Dưỡng ...,2024-11-27,Kem Chống Nắng Mỏng Nhẹ Dưỡng Da Blissberry Gl...,CÔNG DỤNG CHÍNH:Chống nắng phổ rộng với chỉ số...,"THÀNH PHẦN:Chống nắng tối ưu, hiệu quả nhờ các...",CÁCH DÙNG:Bước 1: Lắc đều trước khi sử dụng.Bư...,Thông tin thêmXuất xứ thương hiệuKorea (the Re...
1,https://www.guardian.com.vn/kem-chong-nang-l-o...,3027053,https://www.guardian.com.vn/media/catalog/prod...,Kem Chống Nắng L'Oreal Mỏng Nhẹ Bảo Vệ Tối Đa ...,95,4,299000,379000,L'ORÉAL,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Kem Chống Nắngthương hiệuL’Ore...,2024-11-27,Kem Chống Nắngthương hiệuL’OrealParis UV Defen...,20X kết cấu mỏng nhẹ:kem chống nắng dạng sữa l...,Xem trên bao bì sản phẩm,- Dùng 1 lượng kem vừa đủ và chia đều trên mặt...,Thông tin thêmDành cho daMọi Loại DaXuất xứ th...
2,https://www.guardian.com.vn/gel-chong-nang-cap...,3025476,https://www.guardian.com.vn/media/catalog/prod...,"Gel Chống Nắng Cấp Ẩm, Nâng Tông Chiết Xuất Ý ...",80,1,143000,179000,REIHAKU HATOMUGI,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],"{'Giới Thiệu': 'Gel Chống NắngCấp Ẩm, Nâng Tôn...",2024-11-27,"Gel Chống NắngCấp Ẩm, Nâng Tông Chiết Xuất Ý D...",CÔNG DỤNG CHÍNHChiết xuất hạt ý dĩ cung cấp gi...,"Thành phần:Water, Ethylhexyl Methoxycinnamate,...",CÁCH DÙNG- Thoa kem trước khi tiếp xúc với á...,Thông tin thêmDành cho daMọi Loại DaXuất xứ th...
3,https://www.guardian.com.vn/kem-chong-nang-cho...,3024648,https://www.guardian.com.vn/media/catalog/prod...,Kem Chống Nắng Cho Da Nhạy Cảm Skin1004 Madaga...,95,8,249000,445000,SKIN1004,Thêm vào giỏ hàng,[VoucherG-DN- Giảm 60K cho đơn hàng có địa chỉ...,[[HÀNG TẶNG KHÔNG BÁN] Kem Chống Nắng Dạng Tuý...,{'Giới Thiệu': 'Kem Chống Nắng Cho Da Nhạy Cảm...,2024-11-27,Kem Chống Nắng Cho Da Nhạy Cảm Skin1004 Madaga...,Có khả năng bảo vệ da dưới ánh nắng mặt trời t...,Xem trên bao bì sản phẩm,- Lắc đều trước khi sử dụng- Sử dụng ở bước cu...,Thông tin thêmDành cho daDa Nhạy CảmXuất xứ th...
4,https://www.guardian.com.vn/kem-chong-nang-kie...,3016733,https://www.guardian.com.vn/media/catalog/prod...,Kem Chống Nắng Kiểm Soát Nhờn Không Màu La Roc...,93,3,365000,560000,LA ROCHE-POSAY,Thêm vào giỏ hàng,[VoucherG-DN- Giảm 60K cho đơn hàng có địa chỉ...,"[LA ROCHE-POSAY TÚI TÍM MELA B3 - 50000, [HÀNG...",{'Giới Thiệu': 'Kem Chống Nắng La Roche-Posay ...,2024-11-27,Kem Chống Nắng La Roche-Posay Kiềm Dầu 12H Ant...,"Không gây bết dính, không để lại vệt t...",- Màng lọc độc quyền MEXORYL 400:Màng lọc UV d...,- Thoa kem trước khi tiếp xúc với ánh nắng 20 ...,Thông tin thêmDành cho daMọi Loại DaXuất xứ th...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,https://www.guardian.com.vn/nuoc-tay-trang-pro...,3025187,https://www.guardian.com.vn/media/catalog/prod...,Nước Tẩy Trang Professional Clean Eye & Lip Re...,0,0,99000,229000,CATHY DOLL,Thêm vào giỏ hàng,[VoucherG-LAMQUEN- GIẢM 10% TỐI ĐA 30K CHO ĐƠN...,[],{'Giới Thiệu': 'Nước Tẩy Trang Professional Cl...,2024-11-27,Nước Tẩy Trang Professional Clean Eye & Lip Re...,NaN,NaN,NaN,Thông tin thêmDành cho daMọi Loại DaXuất xứ th...
261,https://www.guardian.com.vn/dau-tay-trang-lam-...,3027315,https://www.guardian.com.vn/media/catalog/prod...,Dầu Tẩy Trang Làm Sạch Cân Bằng Coboté 150Ml,0,0,220000,65000,COBOTÉ,Sắp về hàng,[],[],{'Giới Thiệu': 'THÀNH PHẦN - INGREDIENTS:Dầu d...,2024-11-27,THÀNH PHẦN - INGREDIENTS:Dầu dừa tươi™ chiết x...,NaN,NaN,NaN,Thông tin thêmDành cho daMọi Loại DaXuất xứ th...
2